In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime as dt
import xgboost as xgb
import os

import pickle

from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    ExtraTreesClassifier,
)
from scipy.stats import randint, uniform

## Process weather data

In [ ]:
weather_data = pd.read_csv("../data/processed_weather_data_leuven.csv")

In [ ]:
# Dropping index csv column
weather_data.drop(["Unnamed: 0"], inplace=True, axis=1)

In [ ]:
# Format time stamp
weather_data["time"] = pd.to_datetime(weather_data["time"])
weather_data["date"] = weather_data["time"].dt.date
weather_data["hour"] = weather_data["time"].dt.hour
weather_data["month"] = weather_data["time"].dt.month
weather_data["weekday"] = weather_data["time"].dt.strftime("%a")

In [ ]:
weather_data = (
    weather_data.groupby(["date", "hour", "month", "weekday"]).mean().reset_index()
)

In [ ]:
# Dropping weathercode because signal should be contained in other data + excessive amount of dummies + unseen values
weather_data = weather_data.drop("weathercode", axis=1)

## Process airquality data



In [ ]:
air_quality_data = pd.read_csv("../data/processed_air_quality_data.csv")

In [ ]:
# Dropping index csv column
air_quality_data.drop(["Unnamed: 0"], inplace=True, axis=1)

In [ ]:
# extract from timestamp
air_quality_data["dt"] = pd.to_datetime(air_quality_data["dt"])
air_quality_data["date"] = air_quality_data["dt"].dt.date
air_quality_data["hour"] = air_quality_data["dt"].dt.hour
air_quality_data["month"] = air_quality_data["dt"].dt.month
air_quality_data["weekday"] = air_quality_data["dt"].dt.strftime("%a")

In [ ]:
air_quality_data = (
    air_quality_data.groupby(["date", "hour", "month", "weekday"]).mean().reset_index()
)

## Processing file 40 data, merge all files

In [ ]:
# Noise data
file40 = pd.read_csv("../data/processed_file40_data.csv")

In [ ]:
# Dropping index csv column
file40.drop(["Unnamed: 0"], inplace=True, axis=1)

In [ ]:
# Convert the 'result_timestamp' column to a datetime data type
file40["result_timestamp"] = pd.to_datetime(file40["result_timestamp"])
file40["date"] = file40["result_timestamp"].dt.date
file40["hour"] = file40["result_timestamp"].dt.hour
file40["month"] = file40["result_timestamp"].dt.month
file40["weekday"] = file40["result_timestamp"].dt.strftime("%a")

In [ ]:
file40 = (
    file40.groupby(["object_id", "date", "hour", "month", "weekday"])
    .mean()
    .reset_index()
)

In [ ]:
data_model_v2 = file40.merge(
    air_quality_data,
    how="inner",
    left_on=["date", "hour", "month", "weekday"],
    right_on=["date", "hour", "month", "weekday"],
)

In [ ]:
data_model_v2 = data_model_v2.merge(
    weather_data,
    how="inner",
    left_on=["date", "hour", "month", "weekday"],
    right_on=["date", "hour", "month", "weekday"],
)

In [ ]:
## split train, test data
train_df, val_df = train_test_split(data_model_v2, test_size=0.2, random_state=7)

In [ ]:
data_model_v2

## Process independent variable

In [ ]:
target_variable = [col for col in train_df.columns if col.startswith("laf")]
target_variable

In [ ]:
y_train = train_df[target_variable]

In [ ]:
y_val = val_df[target_variable]

In [ ]:
X_train = train_df.drop(target_variable + ["date"], axis=1)
X_val = val_df.drop(target_variable + ["date"], axis=1)

In [ ]:
one_hot_var = ["hour", "month", "weekday", "object_id"]
numerical_var = [col for col in X_train.columns if col not in one_hot_var]

In [ ]:
t = ColumnTransformer(
    transformers=[
        ("OneHot", OneHotEncoder(handle_unknown="ignore"), one_hot_var),
        ("StandardScaler", StandardScaler(), numerical_var),
    ]
)

# fit the encoder
t.fit(X_train, y_train)

In [ ]:
# Save encoder

pickle.dump(t, open("../model/model_noise_level_file40/encoder.pkl", "wb"))

In [ ]:
# create pandas DataFrame from dense matrix
X_train = pd.DataFrame(t.fit_transform(X_train), columns=t.get_feature_names_out())
X_val = pd.DataFrame(t.transform(X_val), columns=t.get_feature_names_out())

## Predict laf50

This is "exemplatory", laf25/75 are run in the same way

In [ ]:
if os.path.isfile("../model/model_noise_level_file40/laf50_per_hour_dict"):
    print(
        "Params have already been searched and saved, so instead we just load the file"
    )
    params_dict = pickle.load(
        open("../model/model_noise_level_file40/laf50_per_hour_dict", "rb")
    )
else:
    # Define the model parameters
    model_params = {
        "random_forest": {
            "model": RandomForestRegressor(),
            "params": {
                "n_estimators": randint(50, 100),
                "max_depth": randint(3, 50),
                "max_features": ["auto", "sqrt"],
                "min_samples_split": randint(2, 20),
                "min_samples_leaf": randint(1, 10),
                "bootstrap": [True, False],
            },
        },
        "gradient_boosting": {
            "model": GradientBoostingRegressor(),
            "params": {
                "n_estimators": randint(50, 100),
                "learning_rate": uniform(0.01, 0.5),
                "max_depth": randint(1, 10),
                "min_samples_split": randint(2, 20),
                "min_samples_leaf": randint(1, 10),
            },
        },
        "xgboost": {
            "model": xgboost.XGBRegressor(),
            "params": {
                "n_estimators": randint(50, 100),
                "learning_rate": uniform(0.01, 0.5),
                "max_depth": randint(1, 10),
                "min_child_weight": randint(1, 10),
                "gamma": uniform(0, 1),
                "reg_alpha": uniform(0, 1),
                "reg_lambda": uniform(0, 1),
            },
        },
    }

    params_dict = {}

    # Loop through each model in model_params and run RandomizedSearchCV
    for model_name, model_info in model_params.items():
        print("Running RandomizedSearchCV for {}...".format(model_name))

        # Create a RandomizedSearchCV object for the current model
        model = model_info["model"]
        param_dist = model_info["params"]
        random_search = RandomizedSearchCV(
            model,
            param_distributions=param_dist,
            n_iter=10,
            cv=5,
            n_jobs=1,
            random_state=7,
        )

        # Fit the RandomizedSearchCV object to the data
        random_search.fit(X_train, y_train["laf50_per_hour"])

        # Print the best parameters and score
        params_dict[model_name] = random_search.best_params_
        print("Best parameters for {}: ".format(model_name), random_search.best_params_)
        print("Best score for {}: ".format(model_name), random_search.best_score_)
        print("\n")

In [ ]:
# Save optimal param dictionary
pickle.dump(
    params_dict, open("../model/model_noise_level_file40/laf50_per_hour_dict", "wb")
)

In [ ]:
gb_params = params_dict["gradient_boosting"]

gb = GradientBoostingRegressor(**gb_params, random_state=7)

gb.fit(X_train, y_train["laf50_per_hour"])

train_preds = gb.predict(X_train)
val_preds = gb.predict(X_val)

print(
    "Train RMSE:", np.sqrt(mean_squared_error(train_preds, y_train["laf50_per_hour"]))
)
print("Val RMSE:", np.sqrt(mean_squared_error(val_preds, y_val["laf50_per_hour"])))
print("Train MAE:", mean_absolute_error(train_preds, y_train["laf50_per_hour"]))
print("Val MAE:", mean_absolute_error(val_preds, y_val["laf50_per_hour"]))

In [ ]:
rf_params = params_dict["random_forest"]

rf = RandomForestRegressor(**rf_params, random_state=7)

rf.fit(X_train, y_train["laf50_per_hour"])

train_preds = rf.predict(X_train)
val_preds = rf.predict(X_val)

print(
    "Train RMSE:", np.sqrt(mean_squared_error(train_preds, y_train["laf50_per_hour"]))
)
print("Val RMSE:", np.sqrt(mean_squared_error(val_preds, y_val["laf50_per_hour"])))
print("Train MAE:", mean_absolute_error(train_preds, y_train["laf50_per_hour"]))
print("Val MAE:", mean_absolute_error(val_preds, y_val["laf50_per_hour"]))

In [ ]:
import xgboost

xgb_params = params_dict["xgboost"]

xgb = xgboost.XGBRegressor(**xgb_params, random_state=7)

xgb.fit(X_train, y_train["laf50_per_hour"])

train_preds = xgb.predict(X_train)
val_preds = xgb.predict(X_val)


print(
    "Train RMSE:", np.sqrt(mean_squared_error(train_preds, y_train["laf50_per_hour"]))
)
print("Val RMSE:", np.sqrt(mean_squared_error(val_preds, y_val["laf50_per_hour"])))
print("Train MAE:", mean_absolute_error(train_preds, y_train["laf50_per_hour"]))
print("Val MAE:", mean_absolute_error(val_preds, y_val["laf50_per_hour"]))

In [ ]:
plt.scatter(val_preds, y_val["laf50_per_hour"])
plt.xlabel("y pred")
plt.ylabel("y val")

In [ ]:
r2_score(val_preds, y_val["laf50_per_hour"])

In [ ]:
feature_importances = xgb.feature_importances_
sorted_idx = feature_importances.argsort()[::-1]
sorted_importances = feature_importances[sorted_idx[0:30]]
sorted_columns = list(X_train.columns[sorted_idx[0:30]])
plt.barh(sorted_columns, sorted_importances)

In [ ]:
# Saving best model
pickle.dump(xgb, open("../model/model_noise_level_file40/xgb_laf50_per_hour.pkl", "wb"))

## Predict Laf 25/75

In [ ]:
targets = ["laf25_per_hour", "laf75_per_hour"]
model_params_dict = {}
for target in targets:
    if os.path.isfile(f"../model//model_noise_level_file40/{target}_dict"):
        print(
            "Params have already been searched and saved, so instead we just load the file"
        )
        model_params_dict[target] = pickle.load(
            open(f"../model/model_noise_level_file40/{target}_dict", "rb")
        )
    else:
        # Define the model parameters
        model_params = {
            "random_forest": {
                "model": RandomForestRegressor(),
                "params": {
                    "n_estimators": randint(50, 100),
                    "max_depth": randint(3, 50),
                    "max_features": ["auto", "sqrt"],
                    "min_samples_split": randint(2, 20),
                    "min_samples_leaf": randint(1, 10),
                    "bootstrap": [True, False],
                },
            },
            "gradient_boosting": {
                "model": GradientBoostingRegressor(),
                "params": {
                    "n_estimators": randint(50, 100),
                    "learning_rate": uniform(0.01, 0.5),
                    "max_depth": randint(1, 10),
                    "min_samples_split": randint(2, 20),
                    "min_samples_leaf": randint(1, 10),
                },
            },
            "xgboost": {
                "model": xgboost.XGBRegressor(),
                "params": {
                    "n_estimators": randint(50, 100),
                    "learning_rate": uniform(0.01, 0.5),
                    "max_depth": randint(1, 10),
                    "min_child_weight": randint(1, 10),
                    "gamma": uniform(0, 1),
                    "reg_alpha": uniform(0, 1),
                    "reg_lambda": uniform(0, 1),
                },
            },
        }

        params_dict = {}

        # Loop through each model in model_params and run RandomizedSearchCV
        for model_name, model_info in model_params.items():
            print("Running RandomizedSearchCV for {}...".format(model_name))

            # Create a RandomizedSearchCV object for the current model
            model = model_info["model"]
            param_dist = model_info["params"]
            random_search = RandomizedSearchCV(
                model,
                param_distributions=param_dist,
                n_iter=10,
                cv=5,
                n_jobs=1,
                random_state=7,
            )

            # Fit the RandomizedSearchCV object to the data
            random_search.fit(X_train, y_train[target])

            # Print the best parameters and score
            params_dict[model_name] = random_search.best_params_
            print(
                "Best parameters for {}: ".format(model_name),
                random_search.best_params_,
            )
            print("Best score for {}: ".format(model_name), random_search.best_score_)
            print("\n")

        model_params_dict[target] = params_dict
        pickle.dump(
            params_dict, open(f"../model/model_noise_level_file40/{target}_dict.pkl", "wb")
        )

### Running gb, rf, xgb for laf25/laf75 including RMSE/MAE scorings

In [ ]:
gb_models = {}
for target in targets:
    gb_params = model_params_dict[target]["gradient_boosting"]

    gb = GradientBoostingRegressor(**gb_params, random_state=7)

    gb.fit(X_train, y_train[target])

    train_preds = gb.predict(X_train)
    val_preds = gb.predict(X_val)

    print(
        f"Train RMSE of model {target}:",
        np.sqrt(mean_squared_error(train_preds, y_train[target])),
    )
    print(
        f"Val RMSE of model {target}:",
        np.sqrt(mean_squared_error(val_preds, y_val[target])),
    )
    print(
        f"Train MAE of model {target}:",
        mean_absolute_error(train_preds, y_train[target]),
    )
    print(f"Val MAE of model {target}:", mean_absolute_error(val_preds, y_val[target]))
    gb_models[target] = gb

In [ ]:
rf_models = {}
for target in targets:

    rf_params = model_params_dict[target]["random_forest"]

    rf = RandomForestRegressor(**rf_params, random_state=7)

    rf.fit(X_train, y_train[target])

    train_preds = rf.predict(X_train)
    val_preds = rf.predict(X_val)

    print(
        f"Train RMSE of model {target}:",
        np.sqrt(mean_squared_error(train_preds, y_train[target])),
    )
    print(
        f"Val RMSE of model {target}:",
        np.sqrt(mean_squared_error(val_preds, y_val[target])),
    )
    print(
        f"Train MAE of model {target}:",
        mean_absolute_error(train_preds, y_train[target]),
    )
    print(f"Val MAE of model {target}:", mean_absolute_error(val_preds, y_val[target]))
    rf_models[target] = rf

In [ ]:
xgb_models = {}
for target in targets:

    xgb_params = model_params_dict[target]["xgboost"]

    xgb = xgboost.XGBRegressor(**xgb_params, random_state=7)
    xgb.fit(X_train, y_train[target])

    train_preds = xgb.predict(X_train)
    val_preds = xgb.predict(X_val)

    print(
        f"Train RMSE of model {target}:",
        np.sqrt(mean_squared_error(train_preds, y_train[target])),
    )
    print(
        f"Val RMSE of model {target}:",
        np.sqrt(mean_squared_error(val_preds, y_val[target])),
    )
    print(
        f"Train MAE of model {target}:",
        mean_absolute_error(train_preds, y_train[target]),
    )
    print(f"Val MAE of model {target}:", mean_absolute_error(val_preds, y_val[target]))
    xgb_models[target] = xgb

In [ ]:
# Saving best model
import pickle

for target in targets:
    pickle.dump(
        xgb_models[target],
        open(f"../model/model_noise_level_file40/xgb_{target}.pkl", "wb"),
    )